<h1>A simple implementation of Decision Trees to understand the functionality</h1>

<h4><i> A decision tree in simple terms tries to compute all possible choises and the gain we can attain by choosing each of these options. The decision tree uses greedy algorithm and just decides which choice maximises the gain for a split. It chooses the split and repeats this process until we are able to solve our query. </h4>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('C:/Users/Kiran/Desktop/Pending/week8/ML/tennisPlay.csv')

In [3]:
data = data[:14]

In [4]:
data = data[['Day', 'Outlook', 'Temp', 'Humidity', 'Wind', 'PlayTennis']]

In [5]:
data

,Day,Outlook,Temp,Humidity,Wind,PlayTennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


<h4>Please make use of https://en.wikipedia.org/wiki/Entropy_(information_theory) link to learn about entropy function used in the code for calculating information gain</h4>

<h4><i>Creating a simple tree without classes to understand how it works for an iteration </h4>

In [6]:
def get_filtered_level_value(temp, column, target, level, class1, is_verbose):
    
    if(is_verbose):
        print('column = {}\ntarget={}\nlevel = {}\n class1 = {} '.format(column, target, level, class1))
    
    try:
        value = temp[(temp[column] == level) & (temp[target] == class1)].as_matrix().ravel()[-1]
    except IndexError:
        value = 0
    return value

def get_filtered_non_level_value(temp, column, target, level, class1, is_verbose):
    if(is_verbose):
        print('column = {}\ntarget={}\nlevel = {}\n class1 = {} \nrest'.format(column, target, level, class1))
    
    try:
        value = temp[(temp[column] != level) & (temp[target] == class1)].sum().as_matrix().ravel()[-1]
    except IndexError:
        value = 0
    return value

def calc_shannon_entropy(probablity_level_list):
    pi = probablity_level_list[0]
    pj = probablity_level_list[1]
    if(pi == 0 or pj == 0):
        entropy = 0
    else:
        entropy = (- pi * np.log2(pi) - pj * np.log2(pj))
    return entropy

def calc_weighted_entropy(entropy_left, entropy_right, count_total_level, count_total_non_level, count_total):
    return ((count_total_level/count_total) * entropy_left + (count_total_non_level/count_total) * entropy_right)

def get_class_probablity(temp, target, count_total):
    
    classes = temp[target].unique()
    class_probablity_list = []
    for class1 in classes:
        class_probablity_list.append(temp[temp[target] == class1].groupby(target).count().as_matrix().ravel()[-1]/ count_total)
    return class_probablity_list

def calc_information_gain(initial_entropy, wighted_entropy):
    return initial_entropy - wighted_entropy
        
def decision_tree(data, target, exclude_columns = None, distinct_column = None, is_verbose = False):
    columns = list(data.columns)
    columns.remove(exclude_columns)
    columns.remove(target)
    information_gain_list = []
    level_split_list = []
    temp1 = data.copy()
    count_total = len(temp1)
    class_probablity_list = get_class_probablity(temp1, target, count_total)
    initial_entropy = calc_shannon_entropy(class_probablity_list)
    if(is_verbose):
        print('initial_entropy' , initial_entropy)

    for column in columns:
        if(is_verbose):
            print(column)
        temp = pd.DataFrame(temp1[[column, target, distinct_column]].groupby([column, target]).agg(['count'])).copy()
        temp = temp.reset_index()
        levels = temp[column].unique()
        classes = temp[target].unique()
        
        for level in levels:
            
            level_split_list.append(level)
            count_total_level = temp[temp[column] == level].sum().as_matrix().ravel()[-1]
            if(is_verbose):
                print(count_total_level)
            count_total_non_level = temp[temp[column] != level].sum().as_matrix().ravel()[-1]
            probablity_level_list = []
            probablity_non_level_list = []
            
            for class1 in classes:
                
                
                count_level = get_filtered_level_value(temp, column, target, level, class1, is_verbose)
                probablity_level = count_level / count_total_level
                probablity_level_list.append(probablity_level)
                count_non_level = get_filtered_non_level_value(temp, column, target, level, class1, is_verbose)
                probablity_non_level = count_non_level/ count_total_non_level
                probablity_non_level_list.append(probablity_non_level)

            entropy_left = calc_shannon_entropy(probablity_level_list)
            entropy_right = calc_shannon_entropy(probablity_non_level_list)
            wighted_entropy = calc_weighted_entropy(entropy_left, entropy_right, count_total_level, count_total_non_level, count_total)
            infromation_Gain =  calc_information_gain(initial_entropy, wighted_entropy)
            information_gain_list.append(infromation_Gain)
    
    print('\ninformation_gain_list :\n', information_gain_list)
    print('\nlevel_split_list :\n', level_split_list)
    argmax = np.argmax(information_gain_list)
    print('\n\nbest level to split is :{} vs rest, which has an information gain of {}'.format(level_split_list[argmax], information_gain_list[argmax]))
    initial_entropy = information_gain_list[argmax]
#     temp1 = data[[column, target, distinct_column]].groupby([column, target]).agg(['count'])

In [7]:
decision_tree(data, 'PlayTennis', exclude_columns = 'Day', distinct_column = 'Day')


information_gain_list :
 [0.22600024438491684, 0.0031848530446489942, 0.10224356360985076, 0.014956069928972804, 0.025078173505850621, 0.0013397424044413464, 0.15183550136234159, 0.15183550136234159, 0.048127030408269489, 0.048127030408269489]

level_split_list :
 ['Overcast', 'Rain', 'Sunny', 'Cool', 'Hot', 'Mild', 'High', 'Normal', 'Strong', 'Weak']


best level to split is :Overcast vs rest, which has an information gain of 0.22600024438491684


<h4>Based on this we can see that the tree is going to choose the split of Overcast vs rest as the best split as it maximizes the information gain. Other types of measures are also used to make the decision like the gini index.</h4> 